# Current Idea (open to other ideas of course!):

## Micro-Regime Autoencoders (MRAE)

### Problem Statement

Traditional factor models can struggle capturing short-horizon "micro-regimes" in stock price behavior. The Goal of this project is to use a small autoencoder to learn regimes from 20-day *(I just picked a number we can change this obviously)* sliding windows of price/volume features, and then studiy:

- What the regimes look like
- How they transition from one to another
- Whether transitions have predictive power for near-term returns or volatitlity
- Potential comparissons to simple baselines (momentum, volatility)

### Optional Download (You shouldn't need to do this) Please use the link in the readme to get the dataset

In [ ]:
# uncomment to download data. 
# Not needed if you have the .parquet file in the data/historical folder

# !python qualifier\utils\download_stock_data.py

### Load Dataset from file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Set random seeds for reproducibility
np.random.seed(42)

# Data path
PARQUET_PATH = Path("data/historical/all_stocks_historical.parquet")

print(f"Loading data from: {PARQUET_PATH}")
df = pd.read_parquet(PARQUET_PATH)
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn dtypes:")
print(df.dtypes)

Loading data from: data\historical\all_stocks_historical.parquet

DataFrame shape: (89859344, 7)

Column dtypes:
ticker            object
date      datetime64[us]
open             float64
high             float64
low              float64
close            float64
volume           float64
dtype: object


In [2]:
# Standardize and prepare data
print("Available columns:", df.columns.tolist())

# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Sort by ticker and date
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

print("\n" + "=" * 60)
print("Data Summary:")
print("=" * 60)
print(f"Total rows: {len(df):,}")
print(f"Unique tickers: {df['ticker'].nunique():,}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Days covered: {(df['date'].max() - df['date'].min()).days:,} days")
print(f"Date column type: {df['date'].dtype}")

Available columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']

Data Summary:
Total rows: 89,859,344
Unique tickers: 7,144
Date range: 1962-01-02 to 2025-11-13
Days covered: 23,326 days
Date column type: datetime64[us]


In [3]:
# original data with nans
print("\n" + "=" * 60)
print("Original Data:")
print("=" * 60)
display(df.head())

# no nan 
print("\n" + "=" * 60)
print("valid (non-null) data:")
print("=" * 60)

# remove rows with nans
cleaned_df = df.dropna(subset=['close', 'volume'])
display(cleaned_df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']])


Original Data:


,ticker,date,open,high,low,close,volume
0,A,1962-01-02,NaN,NaN,NaN,NaN,NaN
1,A,1962-01-03,NaN,NaN,NaN,NaN,NaN
2,A,1962-01-04,NaN,NaN,NaN,NaN,NaN
3,A,1962-01-05,NaN,NaN,NaN,NaN,NaN
4,A,1962-01-08,NaN,NaN,NaN,NaN,NaN



valid (non-null) data:


,ticker,date,open,high,low,close,volume
9539,A,1999-11-18,27.245214,29.939798,23.951837,26.347021,62546380.0
9540,A,1999-11-19,25.710805,25.748231,23.839568,24.176392,15234146.0
9541,A,1999-11-22,24.737756,26.347021,23.989261,26.347021,6577870.0
9542,A,1999-11-23,25.448832,26.122478,23.951841,23.951841,5975611.0
9543,A,1999-11-24,24.026691,25.112009,23.951841,24.588062,4843231.0
...,...,...,...,...,...,...,...
89859339,ZYXI,2025-11-07,1.030000,1.100000,1.030000,1.040000,133400.0
89859340,ZYXI,2025-11-10,1.030000,1.190000,1.030000,1.180000,185000.0
89859341,ZYXI,2025-11-11,1.160000,1.190000,1.100000,1.160000,67100.0
89859342,ZYXI,2025-11-12,1.160000,1.200000,1.100000,1.100000,73900.0


### Trimming data

In [ ]:
# trim the df_valid to start at the year 2000
cleaned_trimmed_df = cleaned_df[cleaned_df['date'] >= '2000-01-01'].copy()
print("\n" + "=" * 60)
print("Filtered data (Year 2000 onwards):")
print("=" * 60)
print(f"Total rows: {len(cleaned_trimmed_df):,}")
print(f"Date range: {cleaned_trimmed_df['date'].min().date()} to {cleaned_trimmed_df['date'].max().date()}")
display(cleaned_trimmed_df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']].head())

### MVP for testing and debugging (smaller dataset)

In [ ]:
# two-stage universe selection:
# 1. quality filter: remove tickers with insufficient history
# 2. liquidity rank: select top N by average volume from quality-filtered set
# benefit: always get exactly top_n stocks (e.g., 50) with sufficient data

min_days_required = 1000  # ~4 years: enough for windows + train/val/test splits
top_n = 50  # target universe size

print("\n" + "=" * 60)
print(f"Stage 1: Quality Filter (min {min_days_required} trading days)")
print("=" * 60)

# count days per ticker in the full trimmed dataset
rows_per_ticker_all = cleaned_trimmed_df.groupby('ticker').size()
valid_tickers = rows_per_ticker_all[rows_per_ticker_all >= min_days_required].index.tolist()

print(f"Tickers before quality filter: {len(rows_per_ticker_all):,}")
print(f"Tickers after quality filter:  {len(valid_tickers):,}")
print(f"Tickers removed: {len(rows_per_ticker_all) - len(valid_tickers):,}")

# filter to quality tickers only
quality_filtered_df = cleaned_trimmed_df[cleaned_trimmed_df['ticker'].isin(valid_tickers)].copy()

print("\n" + "=" * 60)
print(f"Stage 2: Liquidity Ranking (top {top_n} by avg volume)")
print("=" * 60)

# average volume on quality-filtered stocks
avg_volume_by_ticker = quality_filtered_df.groupby('ticker')['volume'].mean().sort_values(ascending=False)

# select top N from the filtered universe
universe = avg_volume_by_ticker.head(top_n).index.tolist()


***ONLY USE 'data' IN THE NEXT STEPS. IT IS CLEANED AND TRIMMED***

In [ ]:

# create FINAL FINAL dataset
data = quality_filtered_df[quality_filtered_df['ticker'].isin(universe)].copy()
data = data.sort_values(['ticker', 'date']).reset_index(drop=True)

print(f"Selected universe size: {len(universe)} tickers (target: {top_n})")
print(f"✓ All {len(universe)} tickers have >= {min_days_required} days of history")

print("\n" + "-" * 60)
print(f"Top 10 Tickers by Avg Daily Volume (from quality-filtered set):")
print("-" * 60)
for i, (ticker, vol) in enumerate(avg_volume_by_ticker.head(10).items(), 1):
    print(f"  {i:2d}. {ticker:6s} - {vol:>15,.0f} shares/day")

In [ ]:
print("\n" + "=" * 60)
print(f"Final Universe: {len(universe)} stocks")
print("=" * 60)
print(f"Total rows: {len(data):,}")
print(f"Unique tickers: {data['ticker'].nunique()}")
print(f"Date range: {data['date'].min().date()} to {data['date'].max().date()}")


print("\n" + "-" * 60)
print("Data Completeness (per ticker):")
print("-" * 60)
rows_per_ticker = data.groupby('ticker').size()
print(f"  Min:    {rows_per_ticker.min():>5,} days  (>= {min_days_required} guaranteed)")
print(f"  Median: {rows_per_ticker.median():>5,.0f} days")
print(f"  Mean:   {rows_per_ticker.mean():>5,.0f} days")
print(f"  Max:    {rows_per_ticker.max():>5,} days")

print("\n" + "=" * 60)
print("Selected Tickers (sorted by avg volume):")
print(f"  {', '.join(universe)}")

display(data.head(10))

### Feature engineering

In [ ]:
feat_df = data.copy()
feat_df = feat_df.sort_values(['ticker', 'date']).reset_index(drop=True)

# 1. Log price & 1-day log return (uses only t and t-1)
feat_df['log_price'] = np.log(feat_df['close'])
feat_df['log_ret_1d'] = feat_df.groupby('ticker')['log_price'].diff()

# 2. Rolling volatilities of returns (only uses past & current within window)
feat_df['vol_5d'] = (
    feat_df.groupby('ticker')['log_ret_1d']
           .rolling(window=5, min_periods=5)
           .std()
           .reset_index(level=0, drop=True)
)

feat_df['vol_20d'] = (
    feat_df.groupby('ticker')['log_ret_1d']
           .rolling(window=20, min_periods=20)
           .std()
           .reset_index(level=0, drop=True)
)

# 3. Volume z-score with expanding mean/std per ticker (strictly past+current only)
g = feat_df.groupby('ticker')['volume']
exp_mean = g.expanding().mean().reset_index(level=0, drop=True)
exp_std  = g.expanding().std(ddof=0).reset_index(level=0, drop=True)

feat_df['volume_mean_exp'] = exp_mean
feat_df['volume_std_exp']  = exp_std
feat_df['volume_z'] = (feat_df['volume'] - feat_df['volume_mean_exp']) / feat_df['volume_std_exp']

# 4. Intraday range fraction (uses only current bar)
feat_df['range_frac'] = (feat_df['high'] - feat_df['low']) / feat_df['close']

# Keep only needed columns
feature_cols = ['log_ret_1d', 'vol_5d', 'vol_20d', 'volume_z', 'range_frac']

feat_df = feat_df[['ticker', 'date'] + feature_cols]

# Drop rows where features are not fully defined (very early history)
feat_df = feat_df.dropna(subset=feature_cols).reset_index(drop=True)

print("Feature DF shape:", feat_df.shape)
feat_df.head()
